In [1]:
# 데이터 크롤링
import requests
import csv
import json

# API 주소(요리명과 조리 방법)를 통해 데이터 긁어와 csv로 저장하는 코드
url = "http://openapi.foodsafetykorea.go.kr/api/439dfa58a5bd4e228198/COOKRCP01/json/1001/2000"

response = requests.get(url)
data = response.json()

recipes = data['COOKRCP01']['row']

with open('recipe2.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    headers = recipes[0].keys()
    writer.writerow(headers)

    for recipe in recipes:
        writer.writerow(recipe.values())

In [2]:
# 데이터 정제
import pandas as pd

data = pd.read_csv('ingredient.csv', encoding='cp949')

# 대표 식품명 기준 1개만 남기고 나머지 삭제
data = data.drop_duplicates(subset='대표식품명', keep='first')

# 새로운 CSV로 저장
data.to_csv('new_ingredient.csv', index=False, encoding='utf-8')

In [3]:
# 한 번 더 정제
import pandas as pd

data = pd.read_csv('new_ingredient.csv')

# '식품명' 열의 값 정제
data['식품명'] = data['식품명'].str.split('_').str[0] # _과 그 이후 단어 삭제
data['식품명'] = data['식품명'].str.split('(').str[0] # (와 그 이후 단어 삭제

data.to_csv('final_ingredient.csv', index=False, encoding='utf-8')

In [2]:
# 진짜 마지막 정제  
import pandas as pd

data = pd.read_csv('final_ingredient.csv')

# '식품명'에서 ~류 삭제
data['식품명'] = data['식품명'].str.split('류').str[0] # ~류 삭제

data.to_csv('real_final_ingredient.csv', index=False, encoding='utf-8')

In [ ]:
# recipe 정제
import pandas as pd

df = pd.read_csv('recipe.csv')

cut_list = []

for idx, row in df.iterrows():
    if pd.isnull(row['RCP_PARTS_DTLS']):
        continue # NAN 값이면 건너뛰기
    first_char = row['RCP_PARTS_DTLS'][0] # 첫 번째 문자 추출
    if first_char in [':', ')', ']', '(', '[']: # 첫 번째 문자가 특수문자인지 확인
        cut_list.append(first_char) # 첫 번째 문자를 cut_list에 추가
        df.loc[idx, 'RCP_PARTS_DTLS'] = row['RCP_PARTS_DTLS'][1:].strip() # 첫 번째 문자 제거

# 잘라낸 내용을 따로 csv로 저장
cut_df = pd.DataFrame(cut_list, columns=['cut'])
cut_df.to_csv('cut.csv', index=False, encoding='utf-8')

# 나머지 내용을 새로운 csv로 저장
df.to_csv('new_recipe.csv', index=False, encoding='utf-8')


In [36]:
# RCP 방법 열에서 특정 특수문자(?, -)와 그 특수문자와 붙은 단어 삭제하기
import pandas as pd
import re 

df = pd.read_csv('new_recipe.csv')

# 'RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5' 열에서 특수문자(`?`, `-`)와 그 뒤의 모든 문자 삭제
columns = ['RCP_MAIN', 'RCP_SUB1', 'RCP_SUB2', 'RCP_SUB3', 'RCP_SUB4', 'RCP_SUB5']
for column in columns:
    df[column] = df[column].replace(to_replace=r'[?-].*', value='', regex=True)

df.to_csv('recipe_new.csv', index=False, encoding='utf-8')


In [77]:
# 콤마 정제, g 기준으로!
import pandas as pd
import re

# recipe.csv 파일 읽기
df = pd.read_csv('recipe.csv')

# RCP_PARTS_DTLS에 있는 콤마를 전부 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(',', '', x) if pd.notnull(x) else x)

# g 뒤에 콤마를 다 붙임
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub('g', 'g,', x) if pd.notnull(x) else x)

# 문장에 맨 끝에 오는 콤마는 삭제
df['RCP_PARTS_DTLS'] = df['RCP_PARTS_DTLS'].apply(lambda x: re.sub(',$', '', x) if pd.notnull(x) else x)

# 결과를 recipe.csv 파일에 저장
df.to_csv('cut.csv', index=False, encoding='utf-8')


In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# API 요청
url = "https://apis.data.go.kr/B551182/diseaseInfoService/getDissNameCodeList?serviceKey=e4ZK69MusxTk8Rk0s0US%2Bc9uLgmYzyvmC4%2F4cFXVlWm6emK8KIQDH8TA1I8zv173L1YHFn4SU8P9cd217BWT5g%3D%3D&numOfRows=3000&pageNo=1&year=2022&sickCd=A00&sickType=1&medTp=1"

response = requests.get(url)

# BeautifulSoup으로 파싱
soup = BeautifulSoup(response.text, 'lxml')

# 데이터 추출
items = soup.find_all('item')
data = []
for item in items:
    sickCd = item.find('sickcd').text
    sickNm = item.find('sicknm').text
    data.append([sickCd, sickNm])

# 데이터 프레임 생성
df = pd.DataFrame(data, columns=['sickCd', 'sickNm'])

# CSV 파일로 저장
df.to_csv('disease.csv', index=False, encoding='utf-8')

C:\Users\jhy97\AppData\Local\Temp\ipykernel_4392\4098798430.py:10: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, 'lxml')


In [15]:
# 직접 정제해야 하는 값을 판별하기 위해 이미 정제된 g과, '적당량', '약간', ml 등 제외
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('rr.csv')

# 'RCP_PARTS_DTLS' 컬럼의 각 row에 대해 처리
def process_row(row):
    # nan 또는 누락된 데이터 무시
    if pd.isna(row):
        return row
    # 콤마로 분할
    parts = row.split(',')
    # 'g'이 포함된 부분 삭제
    parts = [part for part in parts if 'g' not in part]
    # '적당량' 이 포함된 부분 삭제
    parts = [part for part in parts if '적당량' not in part]
    # '약간' 이 포함된 부분 삭제
    parts = [part for part in parts if '약간' not in part]
    # 'ml' 이 포함된 부분 삭제
    parts = [part for part in parts if 'ml' not in part]
    # 다시 콤마로 결합
    return ', '.join(parts)

# 'RCP_PARTS_DTLS' 컬럼의 각 row에 대해 process_row 함수 적용
df['재료'] = df['재료'].apply(process_row)

# 결과를 ing.csv 파일에 저장
df.to_csv('ing.csv', index=False, encoding='utf-8')

In [21]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('re.csv')

# '재료' 열을 마지막 공백을 기준으로 분할하여 새로운 데이터프레임 생성
new_df = df['재료'].str.rsplit(' ', n=1, expand=True)

# 새로운 데이터프레임을 기존 데이터프레임에 추가
df['재료명'] = new_df[0]
df['재료량'] = new_df[1]

# 결과를 새로운_재료.csv 파일에 저장
df.to_csv('ing.csv', index=False, encoding='utf-8')

In [24]:
# ingre.csv의 '재료명' 테이블과 ingredient.csv의 '식품명' 테이블을 비교하여 존재하지 않는 데이터를 찾아내기
import pandas as pd

# CSV 파일 읽기
ing_df = pd.read_csv('ingre.csv')
ingredient_df = pd.read_csv('ingredient.csv')

# '재료명'과 '식품명'을 기준으로 두 데이터프레임 병합
merged_df = pd.merge(ing_df, ingredient_df, left_on='재료명', right_on='식품명', how='left')

# 데이터프레임에서 검색을 수행하여, '재료명'에는 존재하나 '식품명'에는 존재하지 않는 값이 존재할 경우 그 '재료명' 옆에 X 표시
merged_df['식품명'] = merged_df['식품명'].fillna('X')

# 결과를 새로운_재료.csv 파일에 저장
merged_df.to_csv('new_ingre.csv', index=False, encoding='utf-8')

In [2]:
# 재료 데이터 새로 정제하기
# '대표식품명' 중에서 '식품중분류'를 중복되지 않는 값으로 1개씩 남기기
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('ing.csv')

# '대표식품명'과 '식품중분류'를 기준으로 중복 행 제거
df = df.drop_duplicates(subset=['대표식품명', '식품중분류명'], keep='first')

# 결과를 새로운_재료.csv 파일에 저장
df.to_csv('new_ing.csv', index=False, encoding='utf-8')